In [1]:
import parsl
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
from parsl.app.app import bash_app,python_app
from parsl import File

In [2]:
config = Config(
    executors=[ThreadPoolExecutor()],
    lazy_errors=True
)
parsl.load(config)

In [3]:
## Define Apps ##
@bash_app
def WireDelay(threshIn='', outputs=[], geoDir='', daqId='', fw='', stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/WireDelay.pl %s %s %s %s %s' %(threshIn, outputs[0], geoDir, daqId, fw)

@bash_app
def Combine(inputs=[], outputs=[], stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Combine.pl ' + ' '.join(inputs) + ' ' + str(outputs[0])

@bash_app
def Sort(inputs=[], outputs=[], key1='1', key2='1', stdout='stdout.txt', stderr='stderr.txt'):
    return 'perl ./perl/Sort.pl %s %s %s %s' %(inputs[0], outputs[0], key1, key2)

@bash_app
def Lifetime(inputs=[], outputs=[], gateWidth='1e-5', energyCheck='0', coincidence='1', geoDir='geo/'):
    return 'perl ./perl/SingleChannel.pl %s %s %s %s %s %s' % (inputs[0], outputs[0], gateWidth, 
                                                               energyCheck, coincidence, geoDir)

@bash_app
def Frequency(column, binType, binValue, inputs=[], outputs=[]):
    return 'perl ./perl/Frequency.pl %s %s %s %s %s' % (inputs[0], outputs[0], column, binType, binValue)

In [4]:
## Analysis Parameters ##
# Define what are typically the command-line arguments

# For WireDelay:
thresholdAll = ('files/6119.2016.0104.1.thresh', 'files/6203.2016.0104.1.thresh')
wireDelayData = ('outputs/6119.2016.0104.1.wd_Lifetime', 'outputs/6203.2016.0104.1.wd_Lifetime')
geoDir = './geo' # Also for Lifetime
detectors = ('6119', '6203')
firmwares = ('1.12', '1.12')

# For Combine:
combineOut = 'outputs/combineOut_Lifetime'

# For Sort:
sort_sortKey1 = '2'
sort_sortKey2 = '3'
sortOut = 'outputs/sortOut_Lifetime'

# For Lifetime:
lifetime_gatewidth = '1e-5'
lifetime_energyCheck = '0'
lifetime_coincidence = '1'
lifetimeOut = 'outputs/lifetimeOut_Lifetime'

# For Frequency:
freq_col = '3'
freq_binType = '0'
freq_binValue = '40'
freqOut = 'outputs/freqOut_Lifetime'

In [5]:
## Workflow ##
# 1) WireDelay() takes input Threshold (.thresh) files and converts
#    each to a Wire Delay (.wd) file:
WireDelay_futures = []
for i in range(len(thresholdAll)):
        WireDelay_futures.append(WireDelay(threshIn=thresholdAll[i], outputs=[wireDelayData[i]], 
                                           geoDir=geoDir, daqId=detectors[i], fw=firmwares[i]))

# WireDelay_futures is a list of futures.
# Each future has an outputs list with one output.
WireDelay_outputs = [i.outputs[0] for i in WireDelay_futures]

# 2) Combine() takes the WireDelay files output by WireDelay() and combines
#    them into a single file with name given by --combineOut
Combine_future = Combine(inputs=WireDelay_outputs, outputs=[combineOut])

# 3) Sort() sorts the --combineOut file, producing a new file with name given
#    by --sortOut
Sort_future = Sort(inputs=Combine_future.outputs, outputs=[sortOut], key1=sort_sortKey1, key2=sort_sortKey2)

# 4) Lifetime() does Lifetime things
Lifetime_future = Lifetime(inputs=Sort_future.outputs, outputs=[lifetimeOut], gateWidth=lifetime_gatewidth, 
                           energyCheck=lifetime_energyCheck, coincidence=lifetime_coincidence, geoDir=geoDir)

# 5) Apply the Frequency data transformation to the output of Lifetime
Frequency_future = Frequency(column=freq_col, binType=freq_binType, binValue=freq_binValue, 
                             inputs=Lifetime_future.outputs, outputs=[freqOut])

# Wait for the final result before exiting.
x = Frequency_future.result()

print("Call to EventSearch completed with exit code:", x)    

Call to EventSearch completed with exit code: 0


In [6]:
!head -10 outputs/lifetimeOut_Lifetime

# channelNumb JulianDay PossibleDecayLength(MicroSeconds) StartDecay EndDecay FirstSignalTimeOverThreshold(nanoseconds) SecondSignalTimeoverThreshold(nanoseconds) PossibleDecayNumber
6203.4	2457392	3.99125	0.2461358163203096	0.2461358163665046	22.50	35.00	1
6203.1	2457392	3.99125	0.2461358163203386	0.2461358163665336	38.75	25.00	1
6203.4	2457392	3.96500	0.2461358163206134	0.2461358163665046	22.50	35.00	1
6203.2	2457392	3.99125	0.2461358163212095	0.2461358163674045	62.50	27.50	1
6203.4	2457392	0.58125	0.2466854231135822	0.2466854231203096	26.25	32.50	1
6203.1	2457392	0.58250	0.2466854231136401	0.2466854231203820	36.25	28.75	1
6203.4	2457392	0.55125	0.2466854231139294	0.2466854231203096	20.00	32.50	1
6203.2	2457392	0.58000	0.2466854231144821	0.2466854231211950	40.00	62.50	1
6203.2	2457392	0.53875	0.2466854231149595	0.2466854231211950	26.25	62.50	1


In [7]:
!head -10 outputs/freqOut_Lifetime

0.420896	25	3
0.662709	13	3
0.904522	2	3
1.146335	5	3
1.388147	14	3
1.629960	13	3
1.871773	8	3
2.113586	7	3
2.355398	6	3
2.597211	2	3
